In [3]:
import os
import time
import urllib.request
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
def download_google_images(query, limit=10, download_path="images"):
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--log-level=3")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("useAutomationExtension", False)

    driver = webdriver.Chrome(options=chrome_options)
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": """
        Object.defineProperty(navigator, 'webdriver', {get: () => undefined});
        """
    })

    wait = WebDriverWait(driver, 10)
    driver.get("https://www.google.com/imghp")

    try:
        accept = wait.until(EC.element_to_be_clickable((By.XPATH, '//button/div[normalize-space()="Tout accepter"]')))
        accept.click()
    except:
        pass

    search_box = wait.until(EC.presence_of_element_located((By.NAME, "q")))
    search_box.send_keys(query)
    search_box.send_keys(Keys.RETURN)

    time.sleep(0.5)
    thumbnails = driver.find_elements(By.CSS_SELECTOR, "img.YQ4gaf")

    if not os.path.exists(download_path):
        os.makedirs(download_path)

    count = 0
    for img in thumbnails:
        try:
            # Vérifie que ce n’est pas un logo de site
            parent_classes = img.get_attribute("class")
            if "zr758" in parent_classes:
                continue  # skip les logos

            driver.execute_script("arguments[0].scrollIntoView(true);", img)
            img.click()
            WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "img.iPVvYb"))
            )


            actual_images = driver.find_elements(By.CSS_SELECTOR, "img.iPVvYb")
            for actual_img in actual_images:
                src = actual_img.get_attribute("src")
                if src and "http" in src:
                    filename = os.path.join(download_path, f"{query}_{count}.jpg")
                    urllib.request.urlretrieve(src, filename)
                    print(f"[✓] {filename}")
                    count += 1
                    break
        except Exception as e:
            print("Erreur :", e)

        if count >= limit:
            break

    driver.quit()
    print("Téléchargement terminé.")

download_google_images("girafe", limit=100)

[✓] images\girafe_0.jpg
[✓] images\girafe_1.jpg
[✓] images\girafe_2.jpg
Erreur : Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6F33ACF25+75717]
	GetHandleVerifier [0x00007FF6F33ACF80+75808]
	(No symbol) [0x00007FF6F3178F9A]
	(No symbol) [0x00007FF6F31CF4C6]
	(No symbol) [0x00007FF6F31CF77C]
	(No symbol) [0x00007FF6F3222577]
	(No symbol) [0x00007FF6F31F73BF]
	(No symbol) [0x00007FF6F321F39C]
	(No symbol) [0x00007FF6F31F7153]
	(No symbol) [0x00007FF6F31C0421]
	(No symbol) [0x00007FF6F31C11B3]
	GetHandleVerifier [0x00007FF6F36AD6FD+3223453]
	GetHandleVerifier [0x00007FF6F36A7CA2+3200322]
	GetHandleVerifier [0x00007FF6F36C5AD3+3322739]
	GetHandleVerifier [0x00007FF6F33C69FA+180890]
	GetHandleVerifier [0x00007FF6F33CE0FF+211359]
	GetHandleVerifier [0x00007FF6F33B5274+109332]
	GetHandleVerifier [0x00007FF6F33B5422+109762]
	GetHandleVerifier [0x00007FF6F339BA39+4825]
	BaseThreadInitThunk [0x00007FFBD4DDE8D7+23]
	RtlUserThreadStart [0x00007FFBD5F514FC+44]

[✓] images\girafe_3.jpg
[✓] imag

KeyboardInterrupt: 